# **Necessary Imports**

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout
import matplotlib.pyplot as plt
import cv2

# **Kaggle Setup (.json via Upload)**

In [ ]:
! pip install kaggle --quiet

from google.colab import files

files.upload()

! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json

# **Kaggle Setup (.json from G-Drive)**

In [ ]:
! pip install kaggle --quiet

kaggle_creds_path = "drive/MyDrive/kaggle.json"

from google.colab import drive
drive.mount("/content/drive")

! mkdir ~/.kaggle

! cp {kaggle_creds_path} ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json

drive.flush_and_unmount()

# **Download Dataset - Cats & Dogs**

In [ ]:
dataset = "salader/dogs-vs-cats"

! kaggle datasets download -d {dataset}

! unzip {dataset.split("/")[1] + ".zip"} -d {dataset.split("/")[1]}

# **Download Dataset - Humans**

In [ ]:
dataset = "ashwingupta3012/human-faces"

! kaggle datasets download -d {dataset}

! unzip {dataset.split("/")[1] + ".zip"} -d {dataset.split("/")[1]}

# **Formatting Datasets**

In [ ]:
! pip install split-folders --quiet

import splitfolders

! mkdir -p /content/UnFormattedDataset/Cats
! mkdir -p /content/UnFormattedDataset/Dogs
! mkdir -p /content/UnFormattedDataset/Humans

! cp -r "/content/dogs-vs-cats/train/dogs/." "/content/UnFormattedDataset/Dogs/"
! cp -r "/content/dogs-vs-cats/test/dogs/." "/content/UnFormattedDataset/Dogs/"

! cp -r "/content/dogs-vs-cats/train/cats/." "/content/UnFormattedDataset/Cats/"
! cp -r "/content/dogs-vs-cats/test/cats/." "/content/UnFormattedDataset/Cats/"

! cp -r "/content/human-faces/Humans/." "/content/UnFormattedDataset/Humans/"

splitfolders.ratio("/content/UnFormattedDataset", output="/content/FormattedDataset", ratio=(0.8, 0.2))

# **Generators**

In [ ]:
train_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/FormattedDataset/train',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(256,256)
)

validation_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/FormattedDataset/val',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(256,256)
)

# **Normalizing Image Dataset**

In [ ]:
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)

# **CNN Model**

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(256,256,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax')) # 3 because we have cat, dog & human classes

model.summary()

# **Model Training**

In [ ]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_ds,epochs=10,validation_data=validation_ds)

# **Graphs**

In [ ]:
plt.figure(figsize=(12, 12))
plt.style.use('ggplot')
plt.subplot(2,2,1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Accuracy of the Model')
plt.ylabel('Accuracy', fontsize=12)
plt.xlabel('Epoch', fontsize=12)
plt.legend(['train accuracy', 'validation accuracy'], loc='lower right', prop={'size': 12})

plt.subplot(2,2,2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss of the Model')
plt.ylabel('Loss', fontsize=12)
plt.xlabel('Epoch', fontsize=12)
plt.legend(['train loss', 'validation loss'], loc='best', prop={'size': 12})

# **Model Testing**

In [ ]:
from google.colab import files

uploaded = files.upload()

filename = next(iter(uploaded))

test_img = cv2.imread('/content/{}'.format(filename))

plt.imshow(test_img)

test_img.shape

test_img = cv2.resize(test_img,(256,256))

test_input = test_img.reshape((1,256,256,3))

result = model.predict(test_input)

if (result[0][0] == 1.0):
  print("\nCat {}".format(result[0]))

if (result[0][1] == 1.0):
  print("\nDog {}".format(result[0]))

if (result[0][2] == 1.0):
  print("\nHuman {}".format(result[0]))